In [2]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model

In [3]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")

In [4]:
#df = df_all[np.isfinite(df_all.pace_1)]
df = df_all

In [5]:
paces = df.as_matrix(["pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
df = df.assign(log_means=pd.Series(means).values)
df = df.assign(log_stdevs=pd.Series(stdevs).values)

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [6]:
# Estimate values for all, but only use them if not other value is available
with_history = df[np.isfinite(df_all.pace_3)]
x = with_history.as_matrix(["mean_team_id"])
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)
estimated_log_means = log_means_model.predict(df.mean_team_id.values.reshape(len(df.mean_team_id), 1))
df = df.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)
estimated_log_stdevs = log_stdevs_model.predict(df.mean_team_id.values.reshape(len(df.mean_team_id), 1))
df = df.assign(estimated_log_stdevs=estimated_log_stdevs)



/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [7]:
df

,mean_team_id,teams,name,num_runs,num_valid_times,mean_pace,stdev,pace_1,pace_2,pace_3,pace_4,pace_5,pace_6,log_means,log_stdevs,estimated_log_means,estimated_log_stdevs
0,52.7,BROMMA-VÄLLINGBY SOK/JÄRFÄL;IFK GÖTEBORG,max peter bejmer,3,3,6.337,1.050,5.582,5.608,7.822,NaN,NaN,NaN,1.890567,0.166373,1.944796,0.087202
1,91.0,IFK GÖTEBORG;MARKBYGDENS OK,johan högstrand,4,4,6.198,0.468,5.591,5.902,6.571,6.728,NaN,NaN,1.854745,0.057003,1.962559,0.087890
2,13.2,IFK GÖTEBORG,vetle ruud bråten,4,4,5.750,0.181,5.688,5.636,5.615,6.060,NaN,NaN,1.752109,0.035106,1.926477,0.086493
3,33.2,IFK GÖTEBORG,jonas pilblad,5,5,6.083,0.389,5.978,5.772,6.819,5.756,6.089,NaN,1.807365,0.068628,1.935752,0.086852
4,17.5,IFK GÖTEBORG;OK KÅRE,fredrik edn,2,2,6.333,0.472,5.861,6.805,NaN,NaN,NaN,NaN,1.917658,0.000000,1.928471,0.086570
5,10.5,IFK LIDINGÖ SOK;IFK GÖTEBORG,fredrik bakkman,2,2,5.483,0.028,5.511,5.455,NaN,NaN,NaN,NaN,1.696533,0.000000,1.925225,0.086444
6,16.8,IFK GÖTEBORG,eskil kinneberg,4,4,5.623,0.084,5.611,5.494,5.719,5.668,NaN,NaN,1.727429,0.017203,1.928146,0.086558
7,21.3,KOOVEE,topi anjala,6,6,6.080,0.516,5.621,6.100,5.977,7.164,5.642,5.974,1.816586,0.080558,1.930233,0.086638
8,1.0,KOOVEE,olexander kratov,1,1,5.874,0.000,5.874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.920819,0.086274
9,6.7,VAAJAKOSKEN TERÄ;KOOVEE,jani lakanen,6,6,6.044,0.456,5.874,5.734,6.208,6.717,5.326,6.403,1.801252,0.082382,1.923462,0.086376


In [8]:
# s = sigma and scale = exp(mu).
df['final_log_means'] = np.where(np.isfinite(df["log_means"]) & df["log_means"]>0, df["log_means"], df["estimated_log_means"])
final_log_means = np.exp(df['final_log_means']) 
df['final_log_stdevs'] = np.where(np.isfinite(df["log_stdevs"]) & df["log_stdevs"]>0, df["log_stdevs"], df["estimated_log_stdevs"])
final_log_stdevs = df['final_log_stdevs']

intervals95 = lognorm.interval(0.95, s = final_log_stdevs, scale = final_log_means)
means = lognorm.mean(s = final_log_stdevs, scale = final_log_means)
medians = lognorm.median(s = final_log_stdevs, scale = final_log_means)

In [9]:

df = df.assign(interval95_start = pd.Series( intervals95[0] ).values)
df = df.assign(interval95_end = pd.Series( intervals95[1] ).values)
df = df.assign(p1_in_interval = pd.Series( (df.interval95_start <= df.pace_1) & (df.interval95_end >= df.pace_1) ).values)
df = df.assign(mean_ln = pd.Series(means).values)
df = df.assign(med_ln = pd.Series(medians).values)

In [10]:
df

,mean_team_id,teams,name,num_runs,num_valid_times,mean_pace,stdev,pace_1,pace_2,pace_3,...,log_stdevs,estimated_log_means,estimated_log_stdevs,final_log_means,final_log_stdevs,interval95_start,interval95_end,p1_in_interval,mean_ln,med_ln
0,52.7,BROMMA-VÄLLINGBY SOK/JÄRFÄL;IFK GÖTEBORG,max peter bejmer,3,3,6.337,1.050,5.582,5.608,7.822,...,0.166373,1.944796,0.087202,1.890567,0.166373,4.780198,9.176560,True,6.715426,6.623124
1,91.0,IFK GÖTEBORG;MARKBYGDENS OK,johan högstrand,4,4,6.198,0.468,5.591,5.902,6.571,...,0.057003,1.962559,0.087890,1.854745,0.057003,5.714579,7.145405,False,6.400459,6.390069
2,13.2,IFK GÖTEBORG,vetle ruud bråten,4,4,5.750,0.181,5.688,5.636,5.615,...,0.035106,1.926477,0.086493,1.752109,0.035106,5.383299,6.177514,True,5.770305,5.766750
3,33.2,IFK GÖTEBORG,jonas pilblad,5,5,6.083,0.389,5.978,5.772,6.819,...,0.068628,1.935752,0.086852,1.807365,0.068628,5.327360,6.971798,True,6.108733,6.094365
4,17.5,IFK GÖTEBORG;OK KÅRE,fredrik edn,2,2,6.333,0.472,5.861,6.805,NaN,...,0.000000,1.928471,0.086570,1.917658,0.086570,5.743009,8.063374,True,6.830548,6.805000
5,10.5,IFK LIDINGÖ SOK;IFK GÖTEBORG,fredrik bakkman,2,2,5.483,0.028,5.511,5.455,NaN,...,0.000000,1.925225,0.086444,1.696533,0.086444,4.604825,6.462140,True,5.475420,5.455000
6,16.8,IFK GÖTEBORG,eskil kinneberg,4,4,5.623,0.084,5.611,5.494,5.719,...,0.017203,1.928146,0.086558,1.727429,0.017203,5.439636,5.819101,True,5.627003,5.626170
7,21.3,KOOVEE,topi anjala,6,6,6.080,0.516,5.621,6.100,5.977,...,0.080558,1.930233,0.086638,1.816586,0.080558,5.252450,7.202858,True,6.170816,6.150825
8,1.0,KOOVEE,olexander kratov,1,1,5.874,0.000,5.874,NaN,NaN,...,NaN,1.920819,0.086274,1.920819,0.086274,5.764539,8.084205,True,6.851998,6.826545
9,6.7,VAAJAKOSKEN TERÄ;KOOVEE,jani lakanen,6,6,6.044,0.456,5.874,5.734,6.208,...,0.082382,1.923462,0.086376,1.801252,0.082382,5.154065,7.118654,True,6.077817,6.057227


In [11]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
mean_err = rmse(df.mean_ln, df.pace_1)
med_err = rmse(df.med_ln, df.pace_1)
(mean_err, med_err)

(2.541390016173329, 2.5301453673082346)

In [12]:
np.mean(df.p1_in_interval)

0.6590752242926156

In [13]:
df.to_csv('data/log_normal_estimates_ju.tsv', sep="\t")

In [58]:
runs17 = pd.read_csv('data/csv-results_j2017_ju.tsv', delimiter="\t")


In [63]:
def get_estimate_row(row):
    name = row["competitor-name"].lower()
    
    by_name = df[df['name'] == name]
    by_name_and_colon = df[df['name'].str.contains(name +":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = row["team-name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    #print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.iloc[0]

def get_estimate_params(row):
    estimate_row = get_estimate_row(row)
    final_log_means = estimate_row.final_log_means.values[0]
    final_log_stdevs = estimate_row.final_log_stdevs.values[0]
    #print(f"estimate_row.final_log_means {final_log_means} {final_log_stdevs}")
    return pd.Series({"final_log_means": final_log_means, "final_log_stdevs": final_log_stdevs})

runs17 = runs17[runs17['team-name'].str.contains("Reak")]
estimate_params = runs17.apply(lambda row: get_estimate_params(row), axis=1)
runs17["final_log_means"] = estimate_params.final_log_means
runs17["final_log_stdevs"] = estimate_params.final_log_stdevs

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [82]:
def leg_dist(leg):
    dist = [12.7, 14.2, 12.3, 7.6, 7.9, 10.9, 13.8]
    return dist[leg - 1]

runs17["leg-dist"] = runs17["leg-nro"].apply(lambda nro: leg_dist(nro))


final_means = np.exp(runs17.final_log_means)
intervals95 = lognorm.interval(0.95, s = runs17.final_log_stdevs, scale = final_means)
runs17["start95"] = intervals95[0] * runs17["leg-dist"]
runs17["end95"] = intervals95[1] * runs17["leg-dist"]

#runs17["means"] = lognorm.mean(s = runs17.final_log_stdevs, scale = final_means)
runs17["est_median"] = lognorm.median(s = runs17.final_log_stdevs, scale = final_means) * runs17["leg-dist"]


runs17["leg-time-mins"] = runs17["leg-time"]/60

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [83]:
runs17

,team-id,placement,team-time,team-name,team-nro,leg-nro,emit,leg-time,competitor-name,control-times,final_log_means,final_log_stdevs,start95,end95,means,medians,leg-time-mins,leg-dist,est_median
2996,532,429.0,43713.0,Reaktor Innovations,1,1,1503265.0,6038.0,Jeppe Koivula,1317;1430;1601;1665;1847;2034;2177;2274;2387;2...,2.039449,0.081844,83.149553,114.601623,7.712162,7.686376,100.633333,12.7,97.616975
2997,532,429.0,43713.0,Reaktor Innovations,1,2,1205346.0,7613.0,Mikko Peltonen,1220;1481;1618;1684;1931;2017;2124;2267;2354;2...,2.185339,0.089053,106.063749,150.373443,8.928999,8.893664,126.883333,14.2,126.290028
2998,532,429.0,43713.0,Reaktor Innovations,1,3,1227857.0,5403.0,Joni Freeman,1100;1262;1333;1391;1690;1786;1899;1975;2110;2...,2.158905,0.104116,86.872467,130.656050,8.708726,8.661652,90.050000,12.3,106.538319
2999,532,429.0,43713.0,Reaktor Innovations,1,4,1502405.0,4422.0,Pasi Huhtiniemi,468;712;940;1379;1470;1593;2503;2976;3079;3182...,2.214939,0.077360,59.827419,81.021232,9.188307,9.160854,73.700000,7.6,69.622490
3000,532,429.0,43713.0,Reaktor Innovations,1,5,1230468.0,4596.0,Janne Vaittinen,459;643;923;1201;1456;1548;1701;2594;2804;2945...,2.364244,0.102639,68.713064,102.747562,10.692171,10.636000,76.600000,7.9,84.024400
3001,532,429.0,43713.0,Reaktor Innovations,1,6,1504204.0,7169.0,Tuomas Kareinen,397;670;947;2307;2705;2970;3355;3554;3671;4278...,2.276647,0.170572,76.027528,148.372150,9.886736,9.743952,119.483333,10.9,106.209076
3002,532,429.0,43713.0,Reaktor Innovations,1,7,1230589.0,8470.0,Aki Kaivola,599;777;1139;1905;2010;2241;2423;3300;3874;403...,2.167087,0.095812,99.879675,145.408058,8.772980,8.732804,141.166667,13.8,120.512694
8883,1547,1270.0,63064.0,Reaktor Innovations,2,1,1179469.0,8159.0,Antti-Ville Jokela,1746;2261;2470;2556;2780;2974;3162;3274;3404;3...,2.637824,0.114045,142.010570,222.060822,14.073978,13.982750,135.983333,12.7,177.580922
8884,1547,1270.0,63064.0,Reaktor Innovations,2,2,1221725.0,8973.0,Teemu Matilainen,1671;1964;2077;2152;2409;2507;2725;2887;2980;3...,2.163783,0.107435,100.128689,152.565355,8.754377,8.704000,149.550000,14.2,123.596800
8885,1547,1270.0,63064.0,Reaktor Innovations,2,3,1410723.0,11291.0,Oskari Pirttikoski,2012;2549;4241;4330;4578;4789;4967;5092;5226;6...,2.619483,0.207946,112.338258,253.826014,14.028673,13.728617,188.183333,12.3,168.861992


NameError: name 'runner' is not defined

In [ ]:
x = np.linspace(5, 13, 100)
y = lognorm.pdf(x, s= runner.log_stdevs, scale=np.exp(runner.log_means))

In [ ]:
plt.plot(x, y)

In [ ]:
with_history

### 